<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/games_300_Elitzur_Vaidman_template/games_300_ElitzurVaidman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=1, shots=1)

The [Elitzur-Vaidman bomb tester](https://en.wikipedia.org/wiki/Elitzur%E2%80%93Vaidman_bomb_tester) is a quantum algorithm that evidences one of the weird features of quantum mechanics that come from superposition. This algorithm is a thought experiment that allows us to verify if a bomb is functional without triggering it.

This algorithm is based on beam splitters that are represented by:

$$
U_{BS}(\theta) = R_y(2\theta)
$$

and it works in the following way:

1. A photon travels through the beam splitter
2. We sample the photon and force it to go through one of two paths depending on its state after sampling ($|0\rangle$ or $|1\rangle$). 
3. A bomb is placed on the $|0\rangle$ path. If the bomb goes through it, and the bomb is live, it will explode with probability equal to the probability of the photon sampling returning $|0\rangle$. If the bomb is a dud, the photon can travel freely through the $|0\rangle$ path.
4. Several beam splitters are connected with their outputs conencted to the next beam splitters inputs.
5. At the end of the circuit there are 2 detectors C and D. If there is no functional bomb present, only detector C will beep. However, if there is a genuine bomb, and if the bomb does not explode, either detector may beep. Thus, if detector D beeps at all, we have detected a real bomb without exploding it! But if detector C beeps, we cannot say anything about the bomb.

The goal of this challenge is to calculate the number of bombs that can be certified as functional without detonating them. By adjusting $\theta$ and chagnging the number of beam splitters, this number can be adjusted.

In [ ]:
@qml.qnode(dev)
def is_bomb(angle):
    """Construct a circuit at implements a one shot measurement at the bomb.

    Args:
        - angle (float): transmissivity of the Beam splitter, corresponding
        to a rotation around the Y axis.

    Returns:
        - (np.ndarray): a length-1 array representing result of the one-shot measurement
    """

    # QHACK #
    qml.RY(2 * angle, wires=0)
    # QHACK #

    return qml.sample(qml.PauliZ(0))

In [ ]:
@qml.qnode(dev)
def bomb_tester(angle):
    """Construct a circuit that implements a final one-shot measurement, given that the bomb does not explode

    Args:
        - angle (float): transmissivity of the Beam splitter right before the final detectors

    Returns:
        - (np.ndarray): a length-1 array representing result of the one-shot measurement
    """

    # QHACK #
    qml.RY(2 * angle, wires=0)
    # QHACK #

    return qml.sample(qml.PauliZ(0))

In [ ]:
def simulate(angle, n):
    """Concatenate n bomb circuits and a final measurement, and return the results of 10000 one-shot measurements

    Args:
        - angle (float): transmissivity of all the beam splitters, taken to be identical.
        - n (int): number of bomb circuits concatenated

    Returns:
        - (float): number of bombs successfully tested / number of bombs that didn't explode.
    """

    # QHACK #
    results = {'C': 0, 'D': 0, 'explosion': 0}
    for k in range(0, 10000):
        bomb_exploded = False
        for i in range(n):
            sample = is_bomb(angle)
            if sample == 1:
                bomb_exploded = True
            if bomb_exploded:
                results['explosion'] += 1
                break
        if bomb_exploded:
            continue
        if not bomb_exploded:
            out = bomb_tester(angle)
            if out == 1:
                results['C'] += 1
            else:
                results['D'] += 1

    return results['D'] / (10000 - results['explosion'])

Testing 1.in

In [ ]:
output = simulate(0.78539,1)
print(f"{output}")

Testing 2.in

In [ ]:
output = simulate(1.256637,5)
print(f"{output}")